In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!pip install deepxde

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import deepxde as dde
from deepxde.backend import tf

import matplotlib
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import numpy as np

import imageio

In [ ]:
### CHECK IF WE ARE USING GPU ###
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [ ]:
# NAVIER-STOKES 2D-EQUATION TAYLOR SOLUTION


# https://en.wikipedia.org/wiki/Taylor%E2%80%93Green_vortex
# https://wienkers.com/wp-content/uploads/2019/04/VorflowReport.pdf
# https://github.com/maziarraissi/PINNs/blob/master/main/continuous_time_identification%20(Navier-Stokes)/NavierStokes.py

In [ ]:
PIE = np.pi
print(PIE)

In [ ]:
# SEED
np.random.seed(1234)
tf.set_random_seed(1234)

In [ ]:
# Number of bits (floating points) 64 bit is way slower! Only use it when you know the model works.
dde.config.real.set_float32()
tf.keras.backend.set_floatx('float32')

In [ ]:
# DOMAIN
x_interval = (0.0, 2*PIE)
y_interval = (0.0, 2*PIE)
t_interval = (0.0, 4.0)

Navier-Stokes equation 

In [ ]:
### CONSTANTS ###
rho = 1.0
visc = 0.5


In [ ]:
### DIFFERENTIAL EQUATION ###

def NAVIER_STOKES_equation(x_in, y_out):

  ## VARIABLES ##
  x = x_in[:,0:1]
  y = x_in[:,1:2]
  t = x_in[:,2:3]  

  ## FUNCTIONS ##
        
  # U #
  u = y_out[:,0:1]
    
  u_x = dde.grad.jacobian(y_out, x_in, i=0, j=0)
  u_y = dde.grad.jacobian(y_out, x_in, i=0, j=1)
  u_t = dde.grad.jacobian(y_out, x_in, i=0, j=2)
    
  u_xx = dde.grad.hessian(y_out, x_in, component=0, i=0,j=0)
  u_yy = dde.grad.hessian(y_out, x_in, component=0, i=1,j=1)  

  # V #
  v = y_out[:,1:2]
    
  v_x = dde.grad.jacobian(y_out, x_in, i=1, j=0)
  v_y = dde.grad.jacobian(y_out, x_in, i=1, j=1)
  v_t = dde.grad.jacobian(y_out, x_in, i=1, j=2)
       
  v_xx = dde.grad.hessian(y_out, x_in, component=1, i=0, j=0)
  v_yy = dde.grad.hessian(y_out, x_in, component=1, i=1, j=1)

  # P #
  p = y[:,2:3]
    
  p_x = dde.grad.jacobian(y_out, x_in, i=2, j=0)
  p_y = dde.grad.jacobian(y_out, x_in, i=2, j=1)

  # Continuity equation #
  continuity = u_x + v_y


  # x-momentum and y-momentum #
  eq1 = u_t + u*u_x + v*u_y + 1/rho * p_x - visc*(u_xx + u_yy)
  eq2 = v_t + u*v_x + v*v_y + 1/rho * p_y - visc*(v_xx + v_yy)
      
  return [continuity, eq1, eq2]


In [ ]:
### TAYLOR GREEN VORTEX SOLUTION ###

def Tay_Gre(x,y,t):

    u = np.cos(x)*np.sin(y)*np.exp(-2*visc*t)
    v = -1.0*np.sin(x)*np.cos(y)*np.exp(-2*visc*t)
    P = -rho/4.0 * (np.cos(2.0*x) + np.cos(2.0*y))*np.exp(-4*visc*t)

    return u,v,P

number_steps = 40

x_ax = np.linspace(x_interval[0],x_interval[1],number_steps)
y_ax = np.linspace(y_interval[0],y_interval[1],number_steps)
t_ax = np.linspace(t_interval[0],t_interval[1],number_steps)

X,Y = np.meshgrid(x_ax,y_ax)

plot = 0

number_of_plots = number_steps

if(plot == 1):

    fig, ax = plt.subplots()

    ### ABSOLUTE VECTORFIELD ###

    for i in range(0, number_of_plots, 1):
        t_value = str(round(t_ax[i],3))


        U,V,P = Tay_Gre(X,Y,t_ax[i])
        plt.contourf(X,Y,np.sqrt(U**2 + V**2), vmin = -2, vmax = 2)
        plt.set_cmap("seismic")
        plt.colorbar()
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Absolute Vectorfield at: t = " + t_value)
        plt.show()

    ### QUIVER VECTORFIELD ###

    for i in range(0, number_of_plots, 1):
        t_value = str(round(t_ax[i],3))


        U,V,P = Tay_Gre(X,Y,t_ax[i])
        plt.quiver(X,Y,U,V,scale=15)
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Vectorfield at: t = " + t_value)
        plt.show()

    ### PRESSURE ###

    for i in range(0, number_of_plots, 1):
        t_value = str(round(t_ax[i],3))


        U,V,P = Tay_Gre(X,Y,t_ax[i])
        plt.contourf(X,Y,P, vmin = -1, vmax = 1)
        plt.set_cmap("seismic")
        plt.colorbar()
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Pressure at: t = " + t_value)
        plt.show()




In [ ]:
### CONDITIONS ###

## BOUNDARIES ##

def t0_bound(x_in, on_intial):
  x = x_in[0]
  y = x_in[1]
  t = x_in[2]
  return on_intial or np.isclose(t, t_interval[0])
                         
## BOUNDARY FUNCTIONS ##
                                                   
def zero_velocity(x_in):
   x = x_in[:,0:1]                                     
   y = x_in[:,1:2]                             
   t = x_in[:,2:3]                                               
   return 0

def t0_u_vortex_velocity(x_in):
   x = x_in[:,0:1]                                     
   y = x_in[:,1:2]                             
   t = x_in[:,2:3]
   return np.cos(x)*np.sin(y)*np.exp(-2*visc*t)

def t0_v_vortex_velocity(x_in):
   x = x_in[:,0:1]                                     
   y = x_in[:,1:2]                             
   t = x_in[:,2:3]
   return -1.0*np.sin(x)*np.cos(y)*np.exp(-2*visc*t)

def t0_pressure(x_in):
   x = x_in[:,0:1]                                     
   y = x_in[:,1:2]                             
   t = x_in[:,2:3]
   return -rho/4.0 * (np.cos(2.0*x) + np.cos(2.0*y))*np.exp(-4*visc*t)

def zero_pressure(x_in):
   x = x_in[:,0:1]                                     
   y = x_in[:,1:2]                             
   t = x_in[:,2:3]
   return 0


In [ ]:
### GEOMETRY DEFINITION ###

geom_rectangle= dde.geometry.Rectangle([x_interval[0], y_interval[0]],[x_interval[1], y_interval[1]])

geom_t = dde.geometry.TimeDomain(t_interval[0],t_interval[1])

geomxtime = dde.geometry.GeometryXTime(geom_rectangle,geom_t)


In [ ]:
### IMPLEMENTATION OF CONDITIONS ###

u_t0 = dde.DirichletBC(geomxtime, t0_u_vortex_velocity, t0_bound, component=0)
v_t0 = dde.DirichletBC(geomxtime, t0_v_vortex_velocity, t0_bound, component=1)
p_t0 = dde.DirichletBC(geomxtime, t0_pressure, t0_bound, component=2)

bcs = [

    u_t0, v_t0,

    p_t0,

    ]


In [ ]:
# MODEL SETUP

data = dde.data.TimePDE(geomxtime, NAVIER_STOKES_equation, bcs, num_domain=5000, num_boundary=5000, num_initial=5000, train_distribution='sobol', anchors=None, solution = None, num_test=5000)

FNN_layer = [3] + [100] * 15 + [3]
net = dde.maps.FNN(FNN_layer, "tanh", "Glorot uniform")
#net = dde.maps.ResNet(3,3,40,15, "tanh", "Glorot uniform")
model = dde.Model(data, net)


# Trained over 3 times with lr = 1e-4,1e-5,1e-6 #
model.compile("adam", lr=1e-6)

# CHECKPOINT

checkpointer = dde.callbacks.ModelCheckpoint("model/model.ckpt", verbose=1, save_better_only=True, period=1000)

# EARLY STOP

EarlyStop = dde.callbacks.EarlyStopping(min_delta=1e-9,patience=5000,baseline=None)


In [ ]:
# TRAIN THE NETWORK OR RESTORE OR BOTH #

train_model = 0
restore = 1

best_model_path = "model/best_model/best.ckpt"

best_model_checkpoint = "model/best_model/best.ckpt-50000"

if(restore == 1):
    restore_best_model_path = best_model_checkpoint
else:
    restore_best_model_path = None

if(train_model == 1):
    losshistory, train_state = model.train(epochs=50000, display_every=1000, disregard_previous_best=True, callbacks=[EarlyStop], model_restore_path = restore_best_model_path, model_save_path = best_model_path)
    model.compile("L-BFGS-B")

In [ ]:
# TRAINING AND TEST LOSS

if(train_model == 1):
    dde.saveplot(losshistory, train_state, issave=True, isplot=True)
elif(train_model == 0 and restore == 1):
    model.restore(best_model_checkpoint, verbose=1)


In [ ]:
number_of_plots = number_steps

dpi = 300 # Resolution of the figures

fig, ax = plt.subplots()

filenames_abs_velocity = []


for i in range(0, number_of_plots, 1):
        t_value = str(round(t_ax[i],3))

        filename_abs_velocity = f'{i}.png'
        filenames_abs_velocity.append("figures/abs_velocity/abs_velocity " + filename_abs_velocity)
        
        X_model = np.vstack((np.ravel(X),np.ravel(Y),t_ax[i]*np.ones(np.size(X)))).T

        output = model.predict(X_model)
        U = np.reshape(output[:,0],(-1,number_steps))
        V = np.reshape(output[:,1],(-1,number_steps))
        P = output[:,2]
    
        plt.contourf(X,Y,np.sqrt(U**2 + V**2),vmin = -2,vmax = 2)
        plt.set_cmap("seismic")
        plt.colorbar()
        plt.xlabel("x")
        plt.ylabel("y")
        plt.title("Absolute Vectorfield at: t = " + t_value)
        plt.savefig("figures/abs_velocity/abs_velocity " + filename_abs_velocity, dpi=dpi)
        plt.show()

In [ ]:
number_of_plots = number_steps

fig, ax = plt.subplots()

filenames_velocity = []

for i in range(0, number_of_plots, 1):
  t_value = str(round(t_ax[i],3))

  ### SAVE VELOCITY AS VECTORFIELD ###

  filename_velocity = f'{i}.png'
  filenames_velocity.append("figures/velocity/velocity " + filename_velocity)

  X_model = np.vstack((np.ravel(X),np.ravel(Y),t_ax[i]*np.ones(np.size(X)))).T

  output = model.predict(X_model)
  U = output[:,0]
  V = output[:,1]
  P = output[:,2]

  plt.quiver(X,Y,U,V,scale=15)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Vectorfield at: t = " + t_value)
  plt.savefig("figures/velocity/velocity " + filename_velocity, dpi=dpi)
  plt.show()


In [ ]:
number_of_plots = number_steps

fig, ax = plt.subplots()

filenames_pressure = []

for i in range(0, number_of_plots, 1):
  t_value = str(round(t_ax[i],3))

  ### SAVE PRESSURE AS CONTOURPLOT ###

  filename_pressure =f'{i}.png'
  filenames_pressure.append("figures/pressure/pressure " + filename_pressure)

  X_model = np.vstack((np.ravel(X),np.ravel(Y),t_ax[i]*np.ones(np.size(X)))).T

  output = model.predict(X_model)
  U = output[:,0]
  V = output[:,1]
  P = np.reshape(output[:,2],(-1,number_steps))

  plt.contourf(X,Y,P,vmin=-1,vmax=1)
  plt.set_cmap("seismic")
  plt.colorbar()
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Pressure at: t = " + t_value)
  plt.savefig("figures/pressure/pressure " + filename_pressure, dpi=dpi)
  plt.show()

In [ ]:
with imageio.get_writer("figures/abs_velocity/abs_Vectorfield.gif", mode='I') as writer:
    for filename_abs_velocity in filenames_abs_velocity:
        image = imageio.imread(filename_abs_velocity)
        writer.append_data(image)

In [ ]:
with imageio.get_writer("figures/velocity/Vectorfield.gif", mode='I') as writer:
    for filename_velocity in filenames_velocity:
        image = imageio.imread(filename_velocity)
        writer.append_data(image)
        

In [ ]:
with imageio.get_writer("figures/pressure/Pressurefield.gif", mode='I') as writer:
    for filename_pressure in filenames_pressure:
        image = imageio.imread(filename_pressure)
        writer.append_data(image)


PLOTS OF THE RESIDUAL AND THE L2-RELATIVE-NORM ERROR

In [ ]:
number_of_plots = number_steps

fig, ax = plt.subplots()

filenames_velocity_error = []

l2_u_velocity_error_app = []
l2_v_velocity_error_app = []
l2_uv_velocity_error_app = []

l2_u_relative_velocity_error_app = []
l2_v_relative_velocity_error_app = []
l2_uv_relative_velocity_error_app = []

for i in range(0, number_of_plots, 1):
  t_value = str(round(t_ax[i],3))

  ### SAVE VELOCITY ERROR AS VECTORFIELD ###

  filename_velocity_error = f'{i}.png'
  filenames_velocity_error.append("figures/velocity_error/velocity_error " + filename_velocity_error)

  X_model = np.vstack((np.ravel(X),np.ravel(Y),t_ax[i]*np.ones(np.size(X)))).T

  U_sol,V_sol,P_sol = Tay_Gre(X,Y,t_ax[i])

  U_sol = np.ravel(U_sol)
  V_sol = np.ravel(V_sol)
  P_sol = np.ravel(P_sol)

  output = model.predict(X_model)
  U = output[:,0]
  V = output[:,1]
  P = output[:,2]

  U_norm = np.linalg.norm(U,2)
  V_norm = np.linalg.norm(V,2)
  P_norm = np.linalg.norm(P,2)

  l2_u_velocity_error = np.linalg.norm(U_sol - U,2)
  l2_v_velocity_error = np.linalg.norm(V_sol - V,2)
  l2_uv_velocity_error = np.sqrt(l2_u_velocity_error**2 + l2_v_velocity_error**2)

  l2_u_relative_velocity_error = l2_u_velocity_error/U_norm
  l2_v_relative_velocity_error = l2_v_velocity_error/V_norm
  l2_uv_relative_velocity_error = l2_uv_velocity_error/np.sqrt(U_norm**2 + V_norm**2)

  l2_u_relative_velocity_error_app.append(l2_u_relative_velocity_error)
  l2_v_relative_velocity_error_app.append(l2_v_relative_velocity_error)
  l2_uv_relative_velocity_error_app.append(l2_uv_relative_velocity_error)

  l2_u_velocity_error_app.append(l2_u_velocity_error)
  l2_v_velocity_error_app.append(l2_v_velocity_error)
  l2_uv_velocity_error_app.append(l2_uv_velocity_error)

  print("\nL2-u:", l2_u_velocity_error,"L2-v:", l2_v_velocity_error, "L2-uv:", l2_uv_velocity_error,"")
  print("L2-relative-u:", l2_u_relative_velocity_error,"L2-relative-v:", l2_v_relative_velocity_error, "L2-relative-uv:", l2_uv_relative_velocity_error,"")

  plt.quiver(X,Y,U_sol - U,V_sol - V, scale=15)

  plt.title("Vectorfield error at: t = " + t_value)
  plt.xlabel("x")
  plt.ylabel("y")
  plt.savefig("figures/velocity_error/velocity_error " + filename_velocity_error, dpi=dpi)
  plt.show()

In [ ]:
plt.plot(t_ax,l2_u_velocity_error_app,"--",t_ax,l2_v_velocity_error_app,"--",t_ax,l2_uv_velocity_error_app,"--")
plt.legend(["L2-u", "L2-v", "L2-uv"])
plt.xlabel("t")
plt.ylabel("Absolute L2")
plt.title("L2-error of the velocityfield u & v as a function of time")
plt.savefig("figures/velocity/L2_error", dpi=dpi)
plt.show()

In [ ]:
plt.plot(t_ax,l2_u_relative_velocity_error_app,"--",t_ax,l2_v_relative_velocity_error_app,"--",t_ax,l2_uv_relative_velocity_error_app,"--")
plt.legend(["L2-relative-u","L2-relative-v","L2-relative-uv",])
plt.xlabel("t")
plt.ylabel("Relative L2")
plt.title("L2-relative error of the velocityfield u & v as a function of time")
plt.savefig("figures/velocity/L2_relative_error", dpi=dpi)
plt.show()

In [ ]:
number_of_plots = number_steps

fig, ax = plt.subplots()

filenames_pressure_error = []

l2_pressure_error_app = []
l2_relative_pressure_error_app = []

for i in range(0, number_of_plots, 1):
  t_value = str(round(t_ax[i],3))

  ### SAVE PRESSURE ERROR ###

  filename_pressure_error = f'{i}.png'
  filenames_pressure.append("figures/pressure_error/pressure_error " + filename_pressure_error)

  X_model = np.vstack((np.ravel(X),np.ravel(Y),t_ax[i]*np.ones(np.size(X)))).T

  U_sol, V_sol, P_sol = Tay_Gre(X,Y,t_ax[i])

  #U_sol = np.ravel(U_sol)
  #V_sol = np.ravel(V_sol)
  #P_sol = np.ravel(P_sol)

  output = model.predict(X_model)
  U = output[:,0]
  V = output[:,1]
  P = np.reshape(output[:,2],(-1,number_steps))

  P_norm = np.linalg.norm(P,2)
  l2_pressure_error = np.linalg.norm(P_sol - P,2)
  l2_relative_pressure_error = l2_pressure_error/P_norm

  l2_pressure_error_app.append(l2_pressure_error)
  l2_relative_pressure_error_app.append(l2_relative_pressure_error)

  print("\nL2-p: ", l2_pressure_error)
  print("L2-relative-p", l2_relative_pressure_error)

  plt.contourf(X,Y,P_sol - P, vmin = -2, vmax = 2)
  plt.set_cmap("seismic")
  plt.colorbar()
  plt.xlabel("x")
  plt.ylabel("y")
  plt.title("Pressure error at: t = " + t_value)
  plt.savefig("figures/pressure_error/pressure_error " + filename_pressure_error, dpi=dpi)
  plt.show()

In [ ]:
plt.plot(t_ax, l2_pressure_error_app,"--")
plt.legend(["L2-P"])
plt.xlabel("t")
plt.ylabel("Absolute L2")
plt.title("L2-error of the pressure P as a function of time")
plt.savefig("figures/l2_error/pressure/L2_error", dpi=dpi)
plt.show()

In [ ]:
plt.plot(t_ax, l2_relative_pressure_error_app,"--")
plt.legend(["L2-P"])
plt.xlabel("t")
plt.ylabel("Relative L2")
plt.title("Relative L2-error of the pressure P as a function of time")
plt.savefig("figures/l2_error/pressure/L2_relative_error", dpi=dpi)
plt.show()